In [ ]:
import cv2
import open3d as o3d
import matplotlib.pyplot as plt

from src.stereo_inference import StereoInferenceOnnx
from src.stereo_calibrate import (
    read_stereo_calibration,
    rectify_images,
    read_camera_intrinsics,
)
from src.utils import (
    images_to_tensors,
    visualize_disparity,
    postprocess_disparity,
    create_point_cloud,
)

In [ ]:
# Load stereo calibration parameters
stereo_map_left, stereo_map_right = read_stereo_calibration(
    'camera_configs/stereo_calibration.xml'
)
# Read camera intrinsics
K, distance_between_cameras = read_camera_intrinsics(
    'camera_configs/left_camera_intrinsics.xml'
)
# Load stereo model
stereo_model = StereoInferenceOnnx('models/fs_800_640.onnx')

In [ ]:
# load images
left_image = cv2.imread('data/samples/seat_l.png', cv2.IMREAD_COLOR)
right_image = cv2.imread('data/samples/seat_r.png', cv2.IMREAD_COLOR)

In [ ]:
# Rectify images
left_rectified, right_rectified = rectify_images(
    left_image, right_image, stereo_map_left, stereo_map_right
)
# Preprocess images for the model
left_tensor, right_tensor, scale = images_to_tensors(left_rectified, right_rectified)

# show rectified images
plt.subplot(1, 2, 1)
plt.imshow(left_rectified)
plt.title('Left Rectified Image')
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(right_rectified)
plt.title('Right Rectified Image')
plt.axis('off')
plt.show()

In [ ]:
# Stereo inference
disparity_map = stereo_model(left_tensor, right_tensor)

In [ ]:
# squeeze form 4D to 2D
disparity_map = disparity_map.squeeze(0).squeeze(0)
# Visualize disparity map
disparity_map_viz = visualize_disparity(disparity_map)
# Postprocess disparity map
disparity_map_proc = postprocess_disparity(disparity_map)

# show disparity map with matplotlib
plt.imshow(disparity_map_viz)
plt.axis('off')
plt.show()

In [ ]:
# Create point cloud
pcd = create_point_cloud(
    disparity_map_proc, left_rectified, K, distance_between_cameras, scale=scale
)
o3d.io.write_point_cloud('output/pcd/pointcloud.ply', pcd)